In [1]:
#imports
import pandas as pd
import numpy as np
import re
import time
import sys
pd.set_option('display.max_colwidth', None)
import pickle
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity
from scipy import sparse
from matplotlib import pyplot as plt
import utils as ut

As an alternative to re-making the stored product recommendation dictionaries necessary for my web app to run quickly, I'm going to attempt to modify the ones I already made and remove some portion of the contents. I suspect it will be faster than re-creating them.

In [2]:
#load in new recommender dictionaries (not dataframes)
with open('./compressed/new_vg_rec.pkl', 'rb') as f:
    vg_rec = pickle.load(f)

with open('./pickles/new_movies_rec.pkl', 'rb') as f:
    movies_rec = pickle.load(f)

with open('./compressed/new_books_rec.pkl', 'rb') as f:
    books_rec = pickle.load(f)

In [12]:
len(list(vg_rec['PlayStation 4 500GB Console [Old Model]'])[:50])

50

In [14]:
def shorten_recs(recommender):
    new_rec = {}
    for key, value in recommender.items(): #open dictionary
        new_rec[key] = list(recommender[key])[:50] #new dictionary where values list is a portion of previous size
    return new_rec

In [15]:
new_vg_rec = shorten_recs(vg_rec)

In [17]:
len(new_vg_rec)

15938

In [18]:
len(list(new_vg_rec['PlayStation 4 500GB Console [Old Model]']))

50

Wow... that ran INSTANTLY. Time to run it over the other two dictionaries.

In [19]:
new_movies_rec = shorten_recs(movies_rec)

In [20]:
new_books_rec = shorten_recs(books_rec)

Time to pickle these and see how much smaller they are than the originals.

In [21]:
with open('./compressed/vg_rec_small.pkl', 'wb') as f:
    pickle.dump(new_vg_rec, f)
    
with open ('./compressed/movies_rec_small.pkl', 'wb') as f:
    pickle.dump(new_movies_rec, f)
    
with open('./compressed/books_rec_small.pkl', 'wb') as f:
    pickle.dump(new_books_rec, f)